In [1]:
import pickle
#import sklearn
#from sklearn import metrics
#from sklearn import linear_model
from sklearn.model_selection import train_test_split
import random
import numpy as np
#from wac import WAC
from tqdm.notebook import tqdm
import pandas as pd
#from sklearn import neural_network
#import matplotlib.pyplot as plt
from nltk.corpus import wordnet as wn
from collections import defaultdict as dd
import nltk 
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/crow/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Initiate Datasets

In [2]:
wac2vec = pickle.load(open('ddata/clip.bertvocab.embeddings.513.pkl', 'rb'))
len(wac2vec)

30522

In [3]:
concr_scores = pickle.load(open('ddata/AC_ratings_google3m_koeper_SiW.pkl', 'rb'))
len(concr_scores)

2168990

## ConcreteAbstract Class

In [4]:
class ConcreteAbstract:
    def __init__(self, word_vectors, concr_scores, word_net):
        pass

### Build Abstraction Tree

In [5]:
import warnings

def init_abstraction_tree(min_rating=8):

    wac_words = list(wac2vec.keys())
    wn_words = set(i for i in wn.words())
    wn_wac_words = wn_words & set(wac_words)
    
    concr_scores_subset = concr_scores[concr_scores.RATING >= min_rating]
    leaf_words = [w for w in tqdm(wn_wac_words) if w in concr_scores_subset.index]

    # Get Leaf Synsets...
    leaf_synsets = [wn.synsets(w)[0] for w in leaf_words]

    # Initiate Abstraction Tree
    embeddings = [wac2vec[w] for w in leaf_words]
    data = {
        "SYNSET" : leaf_synsets,
        "WORD" : leaf_words,
        "DIST2LEAF": [0]*len(leaf_synsets),
        "NUM_LEAVES": [1]*len(leaf_synsets),
        "HYPERNYM": [[]]*len(leaf_synsets),
        "HYPONYMS": [[]]*len(leaf_synsets),
        "EMBEDDING" : embeddings
        
    }
    abstraction_tree = pd.DataFrame(data)
    abstraction_tree.index = abstraction_tree.SYNSET

    # Get True Leaf Synsets
    ancestors = set()
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        for s in leaf_synsets:
            #print(set(s.closure(lambda s: s.hypernyms())))
            ancestors = ancestors.union(set(s.closure(lambda s: s.hypernyms())))

    # Remove leaves that are ansestors of other leaves
    true_leaf_synsets = set(leaf_synsets) - ancestors
    ansestor_leaves = set(leaf_synsets) - true_leaf_synsets
    abstraction_tree.drop(ansestor_leaves, inplace=True)
    
    # Remove leaves that have the same Synset
    abstraction_tree.drop_duplicates(subset='SYNSET', inplace=True)
    
    return abstraction_tree

In [6]:
# Grow abstraction tree

def update_dist2leaf(synset, dist):
    h_dist = abstraction_tree.loc[synset, 'DIST2LEAF']
    if h_dist > dist:
        return
    
    abstraction_tree.at[synset, 'DIST2LEAF'] = dist
    
    # go up the hypernym chain and set the distances
    h = abstraction_tree.loc[synset, 'HYPERNYM']
    if len(h) != 0:
        update_dist2leaf(h[0], dist+1)
        

def update_num_leaves(synset, num_leaves):
    orig_num_leaves = abstraction_tree.loc[synset, 'NUM_LEAVES']
    abstraction_tree.at[synset, 'NUM_LEAVES'] = orig_num_leaves + num_leaves
    
    # go up the hypernym chain and update the leaves
    h = abstraction_tree.loc[synset, 'HYPERNYM']
    if len(h) != 0:
        update_num_leaves(h[0], num_leaves)

def grow_abstraction_tree(abstraction_tree):
    """Takes an initial abstraction tree (containing only leaves) and grows
    the rest of the tree."""
    
    synset_list = list(abstraction_tree['SYNSET'])

    for s in tqdm(synset_list):
        h = s.hypernyms()

        if len(h) == 0:
            continue

        h = h[0]

        if h not in abstraction_tree.SYNSET:
            synset_list.append(h)
            abstraction_tree.loc[h] = [
                h,    # SYNSET
                None, # WORD
                0,    # DIST2LEAF
                0,    # NUM_LEAVES
                [],   # HYPERNYM
                [],   # HYPONYMS
                None  # EMBEDDING
            ]

        # Set DIST2LEAF
        s_dist = abstraction_tree.loc[s, 'DIST2LEAF']
        update_dist2leaf(h, s_dist+1)
        #h_dist = abstraction_tree.loc[h, 'DIST2LEAF']
        #print(s)
        #if s_dist >= h_dist:
            #while abstraction_tree.loc[h, 'DIST2LEAF']
            #abstraction_tree.loc[h, 'DIST2LEAF'] = s_dist + 1

        # Set NUM_LEAVES
        s_num_leaves = abstraction_tree.loc[s, 'NUM_LEAVES']
        update_num_leaves(h, s_num_leaves)
        #h_num_leaves = abstraction_tree.loc[h, 'NUM_LEAVES']
        #abstraction_tree.loc[h, 'NUM_LEAVES'] = h_num_leaves + s_num_leaves

        # Add hypernym to synset
        abstraction_tree.loc[s, 'HYPERNYM'] = [h]

        # Add synset to hypernym
        abstraction_tree.loc[h, 'HYPONYMS'].append(s)

In [7]:
# Display abstraction tree
from nltk.tree import Tree

def build_display_tree(df, root_synset, char_limit=20):
    """Display an abstraction tree starting with the root_synset.
    Returns an nltk Tree structure.
    Do not use on big trees!"""
    row = df.loc[root_synset]
    root_name = row['SYNSET'].lemmas()[0].name()[:char_limit]
    if len(row['HYPONYMS']) == 0:
        return root_name
    
    children = [build_display_tree(df, h, char_limit) for h in row['HYPONYMS']]
    
    return Tree(root_name, children)

In [8]:
#abstraction_tree = init_abstraction_tree(9)
abstraction_tree = init_abstraction_tree(7)

  0%|          | 0/14510 [00:00<?, ?it/s]

In [9]:
#abstraction_tree.loc[wn.synset('chordate.n.01'), 'DIST2LEAF']


In [10]:
grow_abstraction_tree(abstraction_tree)

  0%|          | 0/1229 [00:00<?, ?it/s]

In [11]:
#build_display_tree(abstraction_tree, wn.synset('entity.n.01'), char_limit=5)

In [12]:
# Get Positive Synsets

In [13]:
# Number of positive examples we want for each classifier
#pos_count = 3
#neg_count = 9
pos_count = 10
neg_count = 20


def is_leaf(synset):
    if synset not in abstraction_tree.index:
        return False
    return abstraction_tree.loc[synset, 'DIST2LEAF'] == 0

def classifier_capable(synset):
    return abstraction_tree.loc[synset, 'NUM_LEAVES'] >= pos_count

def is_embedding_capable(synset):
    """Return true if the synset is capable of having an embedding."""
    if synset not in abstraction_tree.index:
        return False
    return is_leaf(synset) or classifier_capable(synset)

# dist2leaf gives us the furthest leaf.
# This can be used as a "sort" of way to determine abstraction level
def dist2leaf(synset):
    return abstraction_tree.loc[synset, 'DIST2LEAF']

def is_closer_to_leaf(synset, dist):
    """Return true if the synset is closer to a leaf than dist."""
    return dist2leaf(synset) < dist

def get_hyponyms(synset):
    """Return a list of hyponyms, or itself if there are none."""
    if synset not in abstraction_tree.index:
        return None
    hypos = abstraction_tree.loc[synset, 'HYPONYMS']
    if len(hypos) == 0:
        return [synset]
    else:
        return hypos

def count_embedding_capable(synset_list):
    """Given a list of synsets, returns a count of how many are capable of having an embedding."""
    return sum((is_embedding_capable(s) == True)*1 for s in synset_list)

def expand_hyponym_list(synset_list):
    hypos = []
    for s in synset_list:
        hypos += get_hyponyms(s)
    return hypos

def remove_embedding_incapable(synset_list):
    synset_list = np.array(synset_list)
    return list(synset_list[list(map(is_embedding_capable, synset_list))])

def find_positive_examples(synset, depth=100):
    pos = get_hyponyms(synset)
    for _ in range(depth):
        pos = expand_hyponym_list(pos)
        if count_embedding_capable(pos) >= pos_count:
            return remove_embedding_incapable(pos)
    
    raise Exception("Reached depth of {} without finding enough positive example: {}".format(depth, synset))

In [14]:
find_positive_examples(wn.synset('artifact.n.01'))

[Synset('cone.n.01'),
 Synset('cake.n.01'),
 Synset('raft.n.01'),
 Synset('pill.n.01'),
 Synset('canopy.n.01'),
 Synset('cloak.n.01'),
 Synset('mask.n.01'),
 Synset('canvas.n.01'),
 Synset('tartan.n.01'),
 Synset('silk.n.01'),
 Synset('velvet.n.01'),
 Synset('wool.n.01'),
 Synset('paisley.n.01'),
 Synset('knit.n.01'),
 Synset('linen.n.01'),
 Synset('graffito.n.01'),
 Synset('rope.n.01'),
 Synset('teddy.n.01'),
 Synset('doll.n.01'),
 Synset('lego.n.01'),
 Synset('playhouse.n.01'),
 Synset('menagerie.n.02'),
 Synset('screen.n.01'),
 Synset('gun_muzzle.n.01'),
 Synset('brick.n.01'),
 Synset('asphalt.n.01'),
 Synset('pool.n.01'),
 Synset('fountain.n.01'),
 Synset('balcony.n.01'),
 Synset('airdock.n.01'),
 Synset('building.n.01'),
 Synset('area.n.05'),
 Synset('public_toilet.n.01'),
 Synset('tape.n.01'),
 Synset('conveyance.n.03'),
 Synset('container.n.01'),
 Synset('device.n.01'),
 Synset('equipment.n.01'),
 Synset('furnishing.n.02'),
 Synset('implement.n.01'),
 Synset('consumer_goods.n.01

In [15]:
# Find negative examples...

In [16]:
def find_negative_examples(synset, pos_examples):
    # All synsets
    neg = np.array(abstraction_tree['SYNSET'])
    # Embedding capable synsets
    neg = neg[list(map(is_embedding_capable, neg))]
    # Use only more concrete words (words that are closer to a leaf)
    dist2leaf = abstraction_tree.loc[synset, 'DIST2LEAF']
    neg = neg[list(map(lambda x: is_closer_to_leaf(x, dist2leaf), neg))]
    # neg examples not in positive examples
    neg = set(neg) - set(pos_examples)
    # Don't include yourself
    #neg.remove(synset) # removed by the closer check
    neg_examples = random.sample(list(neg), k=min(neg_count, len(neg)))
    return neg_examples

In [17]:
pos_examples = find_positive_examples(wn.synset('entity.n.01'))

In [18]:
neg_examples = find_negative_examples(wn.synset('entity.n.01'), pos_examples)

In [19]:
neg_examples

[Synset('grillroom.n.01'),
 Synset('keyboard.n.01'),
 Synset('cloak.n.01'),
 Synset('leash.n.01'),
 Synset('hero.n.04'),
 Synset('asphalt.n.01'),
 Synset('scalp.n.01'),
 Synset('whorehouse.n.01'),
 Synset('cow.n.01'),
 Synset('cemetery.n.01'),
 Synset('banjo.n.01'),
 Synset('wasp.n.01'),
 Synset('racetrack.n.01'),
 Synset('shovel.n.01'),
 Synset('limestone.n.01'),
 Synset('pen.n.01'),
 Synset('foundry.n.01'),
 Synset('table.n.01'),
 Synset('terminal.n.01'),
 Synset('bow.n.01')]

In [20]:
def add_positive_negative_examples(synset):
    pos = find_positive_examples(synset)
    abstraction_tree.at[synset, 'POSITIVE'] = pos
    neg = find_negative_examples(synset, pos)
    abstraction_tree.at[synset, 'NEGATIVE'] = neg

In [21]:
def get_classifier_capable():
    """Get a list of synsets capable of having a classifier."""
    return [s for s in abstraction_tree['SYNSET'] if classifier_capable(s)]

In [22]:
def add_pos_neg_all():
    """Add positive and negative examples for each calssifier capable synset."""
    abstraction_tree['POSITIVE'] = [[]]*len(abstraction_tree)
    abstraction_tree['NEGATIVE'] = [[]]*len(abstraction_tree)
    for s in get_classifier_capable():
        add_positive_negative_examples(s)

In [23]:
add_pos_neg_all()

In [24]:
# Build Train/Test datasets

In [25]:
def build_train_test(synset):
    pos_examples = find_positive_examples(synset)
    neg_examples = find_negative_examples(synset, pos_examples)
    X = pos_examples + neg_examples
    y = list(np.ones(len(pos_examples))) + list(np.zeros(len(neg_examples)))
    return train_test_split(X, y, test_size=0.33, stratify=y)

In [26]:
def fill_out_train_test():
    abstraction_tree['X_TRAIN'] = [[]]*len(abstraction_tree)
    abstraction_tree['X_TEST']  = [[]]*len(abstraction_tree)
    abstraction_tree['Y_TRAIN'] = [[]]*len(abstraction_tree)
    abstraction_tree['Y_TEST']  = [[]]*len(abstraction_tree)
    
    synsets = get_classifier_capable()
    for s in synsets:
        X_train, X_test, y_train, y_test  = build_train_test(s)
        abstraction_tree.at[s, 'X_TRAIN'] = X_train
        abstraction_tree.at[s, 'X_TEST']  = X_test
        abstraction_tree.at[s, 'Y_TRAIN'] = y_train
        abstraction_tree.at[s, 'Y_TEST']  = y_test        

In [27]:
fill_out_train_test()

In [28]:
# Train...

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [30]:
def coefs(classifier):
    return classifier.coef_[0]

def build_classifiers():
    abstraction_tree['CLASSIFIER']  = [None]*len(abstraction_tree)
    cc = get_classifier_capable()
    # Start with synsets close to leaves and work our way up to more abstract hypernyms
    cc.sort(key=dist2leaf)
    for ss in cc:
        #print(ss)
        X_train = abstraction_tree.loc[ss, 'X_TRAIN']
        y_train = abstraction_tree.loc[ss, 'Y_TRAIN']
        X_train = list(abstraction_tree.loc[X_train, 'EMBEDDING'])
        lr = LogisticRegression()
        lr.fit(X_train, y_train)
        abstraction_tree.at[ss, 'CLASSIFIER'] = lr
        abstraction_tree.at[ss, 'EMBEDDING'] = coefs(lr)
       
        

In [31]:
build_classifiers()

In [32]:
#abstraction_tree.loc[get_classifier_capable(), 'EMBEDDING']

In [33]:
# Random Baseline (For a single classifier)

In [34]:
def calc_random_baseline():
    pos, tot = 0, 0
    for ss in get_classifier_capable():
        y = abstraction_tree.loc[ss, 'Y_TRAIN']
        pos += sum(y)
        tot += len(y)
        y = abstraction_tree.loc[ss, 'Y_TEST']
        pos += sum(y)
        tot += len(y)
    return pos/tot

In [35]:
calc_random_baseline()

0.4146341463414634

In [36]:
# Most Common Baseline (For a single classifier)

In [37]:
1-calc_random_baseline()

0.5853658536585367

In [38]:
# Evaluate (vs negative examples)

In [39]:
def evaluate_classifiers():
    golds = []
    preds = []
    cc = get_classifier_capable()
    # Evaluate in order (Not stricly needed, but helpful)
    cc.sort(key=dist2leaf)
    for ss in cc:
        X_test = abstraction_tree.loc[ss, 'X_TEST']
        y_test = abstraction_tree.loc[ss, 'Y_TEST']
        X_test = list(abstraction_tree.loc[X_test, 'EMBEDDING'])
        
        c = abstraction_tree.loc[ss, 'CLASSIFIER']
        pred = c.predict_proba(X_test)
        #pred = lr.predict_proba(X_test)
        pred = list(np.argmax(pred, axis=1))
        print(ss)
        print(list(map(int, y_test)))
        print(pred)
        print(metrics.accuracy_score(y_test, pred))
        print()
        preds = preds + pred
        golds = golds + y_test
    return metrics.accuracy_score(golds, preds)

In [40]:
evaluate_classifiers()

Synset('wood.n.01')
[0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1]
0.8181818181818182

Synset('fabric.n.01')
[0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]
0.9090909090909091

Synset('body_of_water.n.01')
[1, 0, 0, 0, 0, 0, 0, 1, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.7

Synset('plant_material.n.01')
[0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
0.7272727272727273

Synset('self-propelled_vehicle.n.01')
[1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0]
[1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0]
1.0

Synset('dish.n.02')
[0, 0, 0, 1, 1, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 1, 0]
0.9

Synset('furniture.n.01')
[1, 0, 1, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.7

Synset('garment.n.01')
[0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
0.7272727272727273

Synset('room.n.01')
[1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
0.9166666666666666

Synset('vessel.n.03')
[0, 1, 0, 1, 0, 0, 0, 0, 1, 0]


0.7588235294117647

In [41]:
# Evaluate (vs distractors)

In [42]:
ss = cc[1]

NameError: name 'cc' is not defined

In [ ]:
ss

In [ ]:
abstraction_tree.loc[ss, 'X_TRAIN']

In [ ]:
X_train = abstraction_tree.loc[ss, 'X_TRAIN']
y_train = abstraction_tree.loc[ss, 'Y_TRAIN']
X_train = list(abstraction_tree.loc[X_train, 'EMBEDDING'])

In [ ]:
lr = LogisticRegression()

In [ ]:
X_train

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
X_test = abstraction_tree.loc[ss, 'X_TEST']
y_test = abstraction_tree.loc[ss, 'Y_TEST']
X_test = list(abstraction_tree.loc[X_test, 'EMBEDDING'])

In [ ]:
pred = lr.predict_proba(X_test)

In [ ]:
pred[1] = [.2, .6]

In [ ]:
np.argmax(pred, axis=1)

In [ ]:
lr.predict_proba(X_test)

In [ ]:
y_test

In [ ]:
help(lr)

In [ ]:
len(lr.coef_[0])

In [ ]:
ss

In [ ]:
abstraction_tree.loc[ss, 'X_TRAIN']

In [ ]:
#build_display_tree(abstraction_tree, wn.synset('entity.n.01'), char_limit=5)

In [ ]:
abstraction_tree